# MovieLens 100k 电影推荐系统 - 第一阶段

## 项目概述
基于MovieLens 100k数据集构建一个基础的电影推荐系统，采用渐进式开发方法：
- **阶段1**: 数据处理 + 协同过滤推荐
- **阶段2**: 加入内容特征 + 混合推荐
- **阶段3**: 多模态特征融合（如有时间）

本notebook专注于第一阶段的实现。


## 1. 环境设置和数据加载


In [ ]:
# 环境配置与依赖安装
# ===========================
# 首次运行项目时请执行此cell安装依赖

# 方法1：自动安装requirements.txt（推荐）
# !pip install -r requirements.txt

# 方法2：快速安装核心依赖
# !pip install pandas numpy scikit-learn matplotlib seaborn ipykernel

# 方法3：使用conda安装（如果使用conda环境）
# !conda install pandas numpy scikit-learn matplotlib seaborn jupyter -y

print("注意: 请根据需要取消注释上面的安装命令并运行")
print("建议: 使用方法1安装完整依赖列表")
print("提示: 安装完成后可以注释掉本cell避免重复安装")


In [ ]:
# 安装项目依赖
import subprocess
import sys

def install_requirements():
    """安装requirements.txt中的依赖包"""
    try:
        print("正在安装项目依赖...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])
        print("依赖安装完成!")
    except subprocess.CalledProcessError as e:
        print(f"依赖安装失败: {e}")
        print("请手动运行: pip install -r requirements.txt")
    except FileNotFoundError:
        print("requirements.txt文件不存在，跳过自动安装")

# 运行依赖安装（首次运行时）
install_requirements()

# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# 设置图表样式
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

print("环境设置完成!")


: 

In [ ]:
# 加载MovieLens 100k数据集
def load_movielens_data(data_path='ml-100k'):
    """
    加载MovieLens 100k数据集
    
    Returns:
        ratings: 评分数据
        movies: 电影数据
        users: 用户数据
    """
    print("正在加载MovieLens 100k数据集...")
    
    # 加载评分数据
    ratings = pd.read_csv(
        f'{data_path}/u.data', 
        sep='\t', 
        names=['user_id', 'movie_id', 'rating', 'timestamp'],
        encoding='latin-1'
    )
    
    # 加载电影数据
    movie_columns = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url'] + \
                   [f'genre_{i}' for i in range(19)]
    movies = pd.read_csv(
        f'{data_path}/u.item', 
        sep='|', 
        names=movie_columns,
        encoding='latin-1'
    )
    
    # 加载用户数据
    users = pd.read_csv(
        f'{data_path}/u.user', 
        sep='|', 
        names=['user_id', 'age', 'gender', 'occupation', 'zipcode'],
        encoding='latin-1'
    )
    
    print("数据加载完成!")
    print(f"   - 评分数据: {len(ratings):,} 条记录")
    print(f"   - 电影数据: {len(movies):,} 部电影")
    print(f"   - 用户数据: {len(users):,} 位用户")
    
    return ratings, movies, users

# 加载数据
ratings, movies, users = load_movielens_data()


In [ ]:
# 数据基本信息
print("数据集基本信息:")
print(f"总评分数: {len(ratings):,}")
print(f"独立用户数: {ratings['user_id'].nunique():,}")
print(f"独立电影数: {ratings['movie_id'].nunique():,}")
print(f"评分范围: {ratings['rating'].min()} - {ratings['rating'].max()}")
print(f"平均评分: {ratings['rating'].mean():.2f}")

# 数据稀疏度
sparsity = 1 - len(ratings) / (ratings['user_id'].nunique() * ratings['movie_id'].nunique())
print(f"数据稀疏度: {sparsity*100:.2f}%")

# 显示前几行数据
print("\n评分数据样例:")
print(ratings.head())

print("\n电影数据样例:")
print(movies[['movie_id', 'title', 'release_date']].head())


In [ ]:
# 创建用户-电影评分矩阵
def create_rating_matrix(ratings_df):
    """
    创建用户-电影评分矩阵
    
    Args:
        ratings_df: 评分数据DataFrame
        
    Returns:
        rating_matrix: pivot后的评分矩阵
    """
    rating_matrix = ratings_df.pivot_table(
        index='user_id', 
        columns='movie_id', 
        values='rating', 
        fill_value=0
    )
    
    print(f"评分矩阵维度: {rating_matrix.shape}")
    print(f"非零评分比例: {(rating_matrix != 0).sum().sum() / (rating_matrix.shape[0] * rating_matrix.shape[1]) * 100:.2f}%")
    
    return rating_matrix

# 创建评分矩阵
rating_matrix = create_rating_matrix(ratings)
print("\n评分矩阵示例:")
print(rating_matrix.iloc[:5, :5])


In [ ]:
# 分割训练集和测试集
train_ratings, test_ratings = train_test_split(
    ratings, 
    test_size=0.2, 
    random_state=42,
    stratify=ratings['rating']  # 按评分分层抽样
)

print("数据分割完成:")
print(f"   训练集: {len(train_ratings):,} 条记录 ({len(train_ratings)/len(ratings)*100:.1f}%)")
print(f"   测试集: {len(test_ratings):,} 条记录 ({len(test_ratings)/len(ratings)*100:.1f}%)")

# 创建训练集评分矩阵
train_matrix = create_rating_matrix(train_ratings)
print("\n训练集评分矩阵创建完成！")


In [ ]:
# 基于用户的协同过滤推荐算法
class SimpleUserBasedCF:
    """
    简化版基于用户的协同过滤推荐系统
    """
    
    def __init__(self, rating_matrix):
        self.rating_matrix = rating_matrix
        self.user_similarity = None
        
    def compute_user_similarity(self):
        """计算用户相似度矩阵"""
        print("计算用户相似度矩阵...")
        self.user_similarity = cosine_similarity(self.rating_matrix)
        print(f"用户相似度矩阵计算完成! 形状: {self.user_similarity.shape}")
    
    def predict_rating(self, user_id, movie_id, k=20):
        """
        预测用户对电影的评分
        
        Args:
            user_id: 用户ID
            movie_id: 电影ID
            k: 使用最相似的k个用户
            
        Returns:
            predicted_rating: 预测评分
        """
        if self.user_similarity is None:
            self.compute_user_similarity()
        
        try:
            user_idx = self.rating_matrix.index.get_loc(user_id)
        except KeyError:
            return self.rating_matrix.mean().mean()
        
        if movie_id not in self.rating_matrix.columns:
            return self.rating_matrix.iloc[user_idx].mean()
        
        # 获取该用户与其他用户的相似度
        user_similarities = self.user_similarity[user_idx]
        
        # 获取对该电影有评分的用户
        movie_ratings = self.rating_matrix[movie_id]
        rated_users = movie_ratings[movie_ratings > 0]
        
        if len(rated_users) == 0:
            return self.rating_matrix.iloc[user_idx].mean()
        
        # 获取相似用户的评分和相似度
        similar_ratings = []
        similar_similarities = []
        
        for similar_user_id in rated_users.index:
            if similar_user_id != user_id:
                similar_user_idx = self.rating_matrix.index.get_loc(similar_user_id)
                similarity = user_similarities[similar_user_idx]
                if similarity > 0:
                    similar_ratings.append(rated_users[similar_user_id])
                    similar_similarities.append(similarity)
        
        if len(similar_ratings) == 0:
            return self.rating_matrix.iloc[user_idx].mean()
        
        # 选择top-k相似用户
        if len(similar_ratings) > k:
            top_k_indices = np.argsort(similar_similarities)[-k:]
            similar_ratings = np.array(similar_ratings)[top_k_indices]
            similar_similarities = np.array(similar_similarities)[top_k_indices]
        
        # 计算加权平均评分
        if np.sum(similar_similarities) == 0:
            return np.mean(similar_ratings)
        
        predicted_rating = np.average(similar_ratings, weights=similar_similarities)
        return np.clip(predicted_rating, 1, 5)
    
    def recommend_movies(self, user_id, n_recommendations=5):
        """为用户推荐电影"""
        if user_id not in self.rating_matrix.index:
            print(f"用户 {user_id} 不存在于训练数据中")
            return []
        
        user_ratings = self.rating_matrix.loc[user_id]
        unrated_movies = user_ratings[user_ratings == 0].index
        
        predictions = []
        for movie_id in unrated_movies[:100]:  # 限制数量以提高速度
            predicted_rating = self.predict_rating(user_id, movie_id)
            predictions.append((movie_id, predicted_rating))
        
        predictions.sort(key=lambda x: x[1], reverse=True)
        return predictions[:n_recommendations]

# 创建并训练模型
user_cf = SimpleUserBasedCF(train_matrix)
user_cf.compute_user_similarity()
print("\n基于用户的协同过滤模型创建完成!")


In [ ]:
# 模型评估
def evaluate_model(model, test_ratings_sample):
    """评估推荐模型性能"""
    print("评估模型性能...")
    
    predictions = []
    actuals = []
    
    for _, row in test_ratings_sample.iterrows():
        user_id = row['user_id']
        movie_id = row['movie_id']
        actual_rating = row['rating']
        
        predicted_rating = model.predict_rating(user_id, movie_id)
        
        predictions.append(predicted_rating)
        actuals.append(actual_rating)
    
    rmse = np.sqrt(mean_squared_error(actuals, predictions))
    mae = mean_absolute_error(actuals, predictions)
    
    print("模型评估完成:")
    print(f"   RMSE: {rmse:.4f}")
    print(f"   MAE: {mae:.4f}")
    
    return rmse, mae

# 使用测试集样本进行评估
test_sample = test_ratings.sample(n=500, random_state=42)  # 使用500个样本
rmse, mae = evaluate_model(user_cf, test_sample)


In [ ]:
# 推荐结果展示
def display_recommendations(user_id, model, movies_df, n_recommendations=5):
    """展示推荐结果"""
    print(f"为用户 {user_id} 生成电影推荐:")
    print("=" * 50)
    
    # 显示用户历史高分电影
    if user_id in train_matrix.index:
        user_ratings = train_matrix.loc[user_id]
        high_rated_movies = user_ratings[user_ratings >= 4].sort_values(ascending=False).head(3)
        
        print(f"\n用户 {user_id} 的历史高分电影 (≥4分):")
        for movie_id, rating in high_rated_movies.items():
            movie_title = movies_df[movies_df['movie_id'] == movie_id]['title'].iloc[0]
            print(f"   • {movie_title}: {rating}分")
    
    # 生成推荐
    print(f"\n推荐的电影:")
    recommendations = model.recommend_movies(user_id, n_recommendations)
    
    if recommendations:
        for i, (movie_id, predicted_rating) in enumerate(recommendations, 1):
            movie_title = movies_df[movies_df['movie_id'] == movie_id]['title'].iloc[0]
            print(f"   {i}. {movie_title} (预测评分: {predicted_rating:.2f})")
    else:
        print("   无法为该用户生成推荐")

# 选择一个测试用户并展示推荐
test_user_id = train_matrix.index[10]  # 选择第11个用户
display_recommendations(test_user_id, user_cf, movies)


In [ ]:
# 创建数据可视化
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('MovieLens 100k 数据分析', fontsize=16, fontweight='bold')

# 1. 评分分布
ratings['rating'].hist(bins=5, ax=axes[0,0], color='skyblue', edgecolor='black')
axes[0,0].set_title('评分分布')
axes[0,0].set_xlabel('评分')
axes[0,0].set_ylabel('频次')

# 2. 用户评分数量分布
user_rating_counts = ratings.groupby('user_id').size()
user_rating_counts.hist(bins=30, ax=axes[0,1], color='lightgreen', edgecolor='black')
axes[0,1].set_title('用户评分数量分布')
axes[0,1].set_xlabel('评分数量')
axes[0,1].set_ylabel('用户数')

# 3. 电影被评分数量分布
movie_rating_counts = ratings.groupby('movie_id').size()
movie_rating_counts.hist(bins=30, ax=axes[1,0], color='orange', edgecolor='black')
axes[1,0].set_title('电影被评分数量分布')
axes[1,0].set_xlabel('被评分次数')
axes[1,0].set_ylabel('电影数')

# 4. 评分矩阵稀疏性可视化（抽样）
sample_matrix = rating_matrix.iloc[:30, :30]
sns.heatmap(sample_matrix, ax=axes[1,1], cmap='Blues', cbar=True)
axes[1,1].set_title('评分矩阵稀疏性 (30x30样本)')
axes[1,1].set_xlabel('电影ID')
axes[1,1].set_ylabel('用户ID')

plt.tight_layout()
plt.show()

print("数据可视化完成!")


In [ ]:
# 第一阶段总结
print("=" * 60)
print("MovieLens 100k 推荐系统 - 第一阶段总结")
print("=" * 60)

print(f"""
完成的工作:
   - 数据加载和预处理
   - 用户-电影评分矩阵构建
   - 基于用户的协同过滤算法实现
   - 模型评估 (RMSE: {rmse:.4f}, MAE: {mae:.4f})
   - 推荐结果展示和可视化

数据集特征:
   • 总评分数: {len(ratings):,}
   • 用户数: {ratings['user_id'].nunique():,}
   • 电影数: {ratings['movie_id'].nunique():,}
   • 数据稀疏度: {sparsity*100:.1f}%
   • 平均评分: {ratings['rating'].mean():.2f}

下一步计划 (第二阶段):
   1. 实现基于物品的协同过滤
   2. 添加矩阵分解方法 (SVD)
   3. 加入电影内容特征 (流派、年份等)
   4. 实现混合推荐系统
   5. 优化算法参数和性能
   6. 解决冷启动问题

技术要点:
   • 协同过滤能够发现用户的隐式偏好
   • 余弦相似度适合处理稀疏数据
   • 需要平衡推荐准确性和多样性
   • 数据稀疏性是推荐系统的主要挑战
""")

print("第一阶段完成! 基础推荐系统框架已建立。")
